In [1]:
# Cell 1: Import libraries
from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd

In [2]:
# Cell 2: Authenticate
flow = InstalledAppFlow.from_client_config(
    {
        "installed": {
            "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
            "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
            "auth_uri": "https://accounts.google.com/o/oauth2/auth",
            "token_uri": "https://oauth2.googleapis.com/token",
            "redirect_uris": ["http://localhost"]
        }
    },
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

credentials = flow.run_local_server(port=8080, prompt='consent')
client = bigquery.Client(project='lablens-476220', credentials=credentials)
print("Connected!")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=LzK3rp626rpSYg70LcAaTQxVaawM3D&prompt=consent&access_type=offline
Connected!


In [3]:
# Cell 3: Test query
test_query = """
SELECT COUNT(*) as count
FROM `physionet-data.mimiciii_clinical.admissions`
"""

df = client.query(test_query).to_dataframe()
print(f"Total admissions: {df['count'][0]}")

/Users/asadullahwaraich/Library/CloudStorage/OneDrive-HigherEducationCommission/Desktop/NEU Fall 2025/MLOps/Project/Data Pipeline/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Total admissions: 58976


In [8]:
# Cell to check available tables
# List all tables in the mimiciii_clinical dataset
tables = client.list_tables('physionet-data.mimiciii_clinical')
print("Available tables:")
for table in tables:
    print(f"  - {table.table_id}")

Available tables:
  - admissions
  - callout
  - caregivers
  - chartevents
  - cptevents
  - d_cpt
  - d_icd_diagnoses
  - d_icd_procedures
  - d_items
  - d_labitems
  - datetimeevents
  - diagnoses_icd
  - drgcodes
  - icustays
  - inputevents_cv
  - inputevents_mv
  - labevents
  - microbiologyevents
  - outputevents
  - patients
  - prescriptions
  - procedureevents_mv
  - procedures_icd
  - services
  - transfers


In [9]:
# Check for other datasets that might contain notes
datasets = list(client.list_datasets('physionet-data'))
for dataset in datasets:
    if 'mimic' in dataset.dataset_id.lower():
        print(f"Dataset: {dataset.dataset_id}")
        

Dataset: mimiciii_clinical
Dataset: mimiciii_derived
Dataset: mimiciii_notes
Dataset: mimiciii_notes_derived


In [10]:
# Cell 4: Corrected query with the right dataset
query = r"""
WITH discharge AS (
    SELECT 
        hadm_id,
        subject_id,
        REGEXP_REPLACE(text, r'\[\*\*[^\]]*\*\*\]', '') AS cleaned_text,
        LENGTH(text) as text_length
    FROM `physionet-data.mimiciii_notes.noteevents`
    WHERE category = 'Discharge summary'
    LIMIT 5000
),
labs AS (
    SELECT 
        le.hadm_id,
        STRING_AGG(
            CONCAT(d.label, ': ', CAST(le.value AS STRING), ' ', le.valueuom,
            CASE WHEN le.flag = 'abnormal' THEN ' (!)' ELSE '' END), '; '
            LIMIT 20
        ) AS lab_summary,
        SUM(CASE WHEN le.flag = 'abnormal' THEN 1 ELSE 0 END) AS abnormal_count
    FROM `physionet-data.mimiciii_clinical.labevents` le
    JOIN `physionet-data.mimiciii_clinical.d_labitems` d ON le.itemid = d.itemid
    GROUP BY le.hadm_id
)
SELECT 
    d.*,
    l.lab_summary,
    l.abnormal_count
FROM discharge d
LEFT JOIN labs l USING(hadm_id)
"""

print("Running query on correct dataset...")
df = client.query(query).to_dataframe()
print(f"✓ Loaded {len(df)} discharge summaries with lab data")
print(f"✓ Columns: {df.columns.tolist()}")
print(f"✓ Average text length: {df['text_length'].mean():.0f} characters")

Running query on correct dataset...
✓ Loaded 5000 discharge summaries with lab data
✓ Columns: ['hadm_id', 'subject_id', 'cleaned_text', 'text_length', 'lab_summary', 'abnormal_count']
✓ Average text length: 9558 characters


In [11]:
# Save the data
import os

os.makedirs('data/raw', exist_ok=True)
df.to_csv('data/raw/mimic_discharge_labs.csv', index=False)
print(f"✓ Data saved to data/raw/mimic_discharge_labs.csv")
print(f"✓ File size: {os.path.getsize('data/raw/mimic_discharge_labs.csv') / 1024 / 1024:.2f} MB")

✓ Data saved to data/raw/mimic_discharge_labs.csv
✓ File size: 43.97 MB


In [12]:
# Enhanced query with demographics for bias detection
query = r"""
WITH discharge AS (
    SELECT 
        n.hadm_id,
        n.subject_id,
        REGEXP_REPLACE(n.text, r'\[\*\*[^\]]*\*\*\]', '') AS cleaned_text,
        LENGTH(n.text) as text_length
    FROM `physionet-data.mimiciii_notes.noteevents` n
    WHERE n.category = 'Discharge summary'
    LIMIT 5000
),
demographics AS (
    SELECT 
        p.subject_id,
        p.gender,
        a.ethnicity,
        a.insurance,
        a.language,
        a.marital_status,
        a.admission_type,
        DATETIME_DIFF(a.admittime, p.dob, YEAR) as age_at_admission
    FROM `physionet-data.mimiciii_clinical.patients` p
    JOIN `physionet-data.mimiciii_clinical.admissions` a 
        ON p.subject_id = a.subject_id
),
labs AS (
    SELECT 
        le.hadm_id,
        STRING_AGG(
            CONCAT(d.label, ': ', CAST(le.value AS STRING), ' ', le.valueuom,
            CASE WHEN le.flag = 'abnormal' THEN ' (!)' ELSE '' END), '; '
            LIMIT 20
        ) AS lab_summary,
        COUNT(*) as total_labs,
        SUM(CASE WHEN le.flag = 'abnormal' THEN 1 ELSE 0 END) AS abnormal_count
    FROM `physionet-data.mimiciii_clinical.labevents` le
    JOIN `physionet-data.mimiciii_clinical.d_labitems` d ON le.itemid = d.itemid
    GROUP BY le.hadm_id
),
diagnoses AS (
    SELECT 
        hadm_id,
        COUNT(*) as diagnosis_count,
        STRING_AGG(icd9_code, ', ' LIMIT 10) as top_diagnoses
    FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
    GROUP BY hadm_id
)
SELECT 
    d.*,
    dem.gender,
    dem.ethnicity,
    dem.age_at_admission,
    dem.insurance,
    dem.language,
    dem.admission_type,
    l.lab_summary,
    l.total_labs,
    l.abnormal_count,
    diag.diagnosis_count,
    diag.top_diagnoses
FROM discharge d
LEFT JOIN demographics dem ON d.subject_id = dem.subject_id
LEFT JOIN labs l ON d.hadm_id = l.hadm_id
LEFT JOIN diagnoses diag ON d.hadm_id = diag.hadm_id
"""

print("Running comprehensive query with demographics...")
df_complete = client.query(query).to_dataframe()
print(f"✓ Loaded {len(df_complete)} records with full demographics")
print(f"✓ Columns: {df_complete.columns.tolist()}")

# Save complete dataset
df_complete.to_csv('data/raw/mimic_complete_with_demographics.csv', index=False)
print("✓ Saved complete dataset for bias detection")

Running comprehensive query with demographics...


/Users/asadullahwaraich/Library/CloudStorage/OneDrive-HigherEducationCommission/Desktop/NEU Fall 2025/MLOps/Project/Data Pipeline/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✓ Loaded 9996 records with full demographics
✓ Columns: ['hadm_id', 'subject_id', 'cleaned_text', 'text_length', 'gender', 'ethnicity', 'age_at_admission', 'insurance', 'language', 'admission_type', 'lab_summary', 'total_labs', 'abnormal_count', 'diagnosis_count', 'top_diagnoses']
✓ Saved complete dataset for bias detection
